In [ ]:
##Round 1 cleaning complete. v1 decisions: keep all nan as new feature/value; no skew or outlier action has been taken
##Round 1 exploration complete. (a) Distribution of each feature explored (b) feature-target and feature-feature relationships explored 

##Round 1 modelling Plan: make one model for each notebook, try to do systematic tuning ... take best few models (?) or is it too soon?
##then go back and do feature engineering and feature selection based on these base-line models
##Build pipe-line: dummies, scaling, tuning ...

##next steps:
##
##Flask API Deployment
##build small webpage interaction


### Tuning of KNN 

Main parameters: k = num_neighbors, and metric

In [ ]:
## following https://machinelearningmastery.com/hyperparameters-for-classification-machine-learning-algorithms/

In [ ]:
##brief review of KNN vs K-means ... 
##K-means (unspervised) has no labels, just feature space ... you create k-labels (choice of k is tricky and important)
##then run thru k-means clustering algorithm to assign k-labels (clusters) to the data 
##b
##KNN (supervised) is simpler because data already comes with p-labels (the role of k is different here) 
##in the training data on the test data you simply look at the k-nearet neighors (of training set) and take majority vote 
##on what label to assign to new point. The choice of k is important and the metric can change the outcomes.

In [2]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt

from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, RobustScaler, QuantileTransformer
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.decomposition import PCA 
from sklearn.linear_model import Ridge
from sklearn.preprocessing import OneHotEncoder

from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, f1_score
import seaborn as sns

import pickle

#sns.set()
sns.set(rc={'figure.figsize':(8,5)})



In [3]:
## cleaning tool


def perc_null(X):
    
    total = X.isnull().sum().sort_values(ascending=False)
    data_types = X.dtypes
    percent = (X.isnull().sum()/X.isnull().count()).sort_values(ascending=False)

    missing_data = pd.concat([total, data_types, percent], axis=1, keys=['Total','Type' ,'Percent'])
    return missing_data


In [4]:
path = "/Users/joejohns/Downloads/data_loans.csv"

data = pd.read_csv(path) 
print(data.shape)
data.head(3)

(614, 13)


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y


## data processing from round 1


In [5]:
perc_null(data)
    

,Total,Type,Percent
Credit_History,50,float64,0.081433
Self_Employed,32,object,0.052117
LoanAmount,22,float64,0.035831
Dependents,15,object,0.024430
Loan_Amount_Term,14,float64,0.022801
Gender,13,object,0.021173
Married,3,object,0.004886
Loan_ID,0,object,0.000000
Education,0,object,0.000000
ApplicantIncome,0,int64,0.000000


In [6]:
data.rename(columns = {'ApplicantIncome':'Applicant_Income', 'CoapplicantIncome': 'Coapplicant_Income', 'LoanAmount':'Loan_Amount'}, inplace = True)

In [7]:
data.columns

Index(['Loan_ID', 'Gender', 'Married', 'Dependents', 'Education',
       'Self_Employed', 'Applicant_Income', 'Coapplicant_Income',
       'Loan_Amount', 'Loan_Amount_Term', 'Credit_History', 'Property_Area',
       'Loan_Status'],
      dtype='object')

In [8]:
target = ['Loan_Status']
ID_feats = ['Loan_ID',] 

#feature types:

num_feats = ['Applicant_Income', 'Coapplicant_Income', 'Loan_Amount', 'Loan_Amount_Term',]

ord_cat_feats = [ 'Dependents', 'Education', ]
nom_cat_feats = ['Gender', 'Married', 'Self_Employed', 'Credit_History', 'Property_Area',]




#create new category values
make_new_catval_unk = [  'Credit_History', 'Self_Employed',  'Dependents', 'Gender', 'Married', ] 

#create new category

make_new_cat_unk = [ 'Loan_Amount', 'Loan_Amount_Term',]

#if any value is unknown for either feature we make it 'yes')
#cat = 'missing_data'; in general we can have several lists of related variables and make a new cat for each list; 
#name of cat should be specified

#imputing

impute_mean_feats = ['Loan_Amount']
impute_freq_feats = [ 'Loan_Amount_Term']


#scaling
scaling_feats = ord_cat_feats+num_feats

#dummies
dummies_feats = nom_cat_feats #default setting of this variable ... 
#ordinal_encode
ord_feats = ord_cat_feats

#rest of categoricals get new catgory value 'unk' ... it is not very predictive in most cases ...

In [9]:


for item in ['0', '1', '2', '3+']:
    if item != '3+':
        i = int(item)
    else:
        i = 3
    data.loc[data['Dependents'] == item,'Dependents']  = i


In [10]:

data.loc[data["Loan_Status"] == 'Y', "Loan_Status"] = 1
data.loc[data["Loan_Status"] == 'N', "Loan_Status"] = 0



In [11]:

data.loc[data['Credit_History'] ==0.0, 'Credit_History'] = 'No'
data.loc[data['Credit_History'] == 1.0, 'Credit_History'] = 'Yes'  #I use capitalized "Yes", "No" to be conistent with their other entries

In [12]:
data.loc[data['Education']=='Not Graduate', "Education"] = 0
data.loc[data['Education']=='Graduate', "Education"] = 1

In [13]:

#we don't want to do this for cat_ord_feats, because those will be numeric; we do impute_most_freq instead
for cat in nom_cat_feats:
    if data[cat].isnull().sum()>0: 
        data.loc[data[cat].isnull(), cat] = 'unk'

In [14]:
## These are the two numerical features with NaN .. here we set up two new categories to capture these NaN since
## they seem decently predictive. ... Now what to do with the NaN values? I guess just do mean and most-common


#data['Loan_Amount_Term_unk'] = 'No' #set default to 'No' then set a few to be 'Yes'

#create one new categorical for both (if either or both is unknown)
data['missing_data'] = 'No'  
data.loc[(data['Loan_Amount_Term'].isnull()) , 'missing_data'] = 'Yes'
data.loc[(data['Loan_Amount'].isnull()) , 'missing_data'] = 'Yes'
data.loc[(data['Dependents'].isnull()) , 'missing_data'] = 'Yes'



In [15]:
#update nom_cat_feats (no null values, ok)

nom_cat_feats+= ['missing_data']

#### Done with cleaning categorical features ... 
note: we have made preliminary decision to make all NaN in categorical columns into new cat_val ... \
we'll see how this works out ... in some cases it seems to be a good predictor.

In phase 2 investigation, below, we will investigate the naive predictive strength of all the \
categorical features (including NaN as 'unk')


In [16]:
##baseline models
from sklearn.model_selection import train_test_split # For train/test splits
from sklearn.neighbors import KNeighborsClassifier # The k-nearest neighbor classifier
from sklearn.feature_selection import VarianceThreshold # Feature selector
from sklearn.pipeline import Pipeline # For setting up pipeline
# Various pre-processing steps
from sklearn.preprocessing import Normalizer, StandardScaler, MinMaxScaler, PowerTransformer, MaxAbsScaler, LabelEncoder

from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import RandomizedSearchCV

from sklearn.model_selection import GridSearchCV # For optimization


In [17]:

freq_imp = SimpleImputer(missing_values=np.nan,strategy="most_frequent")
mean_imp = SimpleImputer(missing_values=np.nan, strategy="mean")
scaler = StandardScaler()
dummies = OneHotEncoder(handle_unknown='ignore')

#other options:
#1. create new class that has transform and fit and uses make_dummies instead 
#2. put a FunctionTransformer() in the pipeline 



'''cat_impute_most_freq_feats = cat_ord_feats.copy()
cat_scale_feats = cat_ord_feats.copy()
cat_dummies_feats = cat_nom_feats.copy() 
num_scale_feats = num_feats.copy()
mean_imp_feats = [  'Applicant_Income', 'Coapplicant_Income', 'Loan_Amount']
impute_most_freq_feats = ['Loan_Amount_Term',]

num_mean_imp_feats = [  'Applicant_Income', 'Coapplicant_Income', 'Loan_Amount']
num_impute_most_freq_feats = ['Loan_Amount_Term',]
'''


#imputing

impute_mean_feats = ['Loan_Amount',]
impute_freq_feats = [ 'Loan_Amount_Term', 'Dependents']


#scaling
scaling_feats = ord_cat_feats+num_feats

#dummies
dummies_feats = nom_cat_feats #default setting of this variable ... 
#ordinal_encode
ord_feats = ord_cat_feats  #done manually at this point

preprocess = ColumnTransformer([
    ('most_freq_imp', freq_imp, impute_freq_feats),
    ('mean_imp', mean_imp, impute_mean_feats), #only one numerical feature has NaN right now
    ('scaler', scaler, scaling_feats),
    ('dummies', dummies, dummies_feats), 
])

pipeline = Pipeline([
    ('preprocess', preprocess),
    ('knn', KNeighborsClassifier())
])

In [ ]:
#let me now follow https://machinelearningmastery.com/modeling-pipeline-optimization-with-scikit-learn/
#which does cv = 3 instead of cv = stratififedkfold( ... )

model_feats = ord_cat_feats+nom_cat_feats+num_feats

y = data.loc[:, target]
X= data.loc[:, model_feats]

# Split the data into test and train
X_train, X_test, y_train, y_test = train_test_split(
    X,  
    y, 
    test_size=1/3,
    random_state=0)
 
print(X_train.shape)
print(X_test.shape)

#np.array(X)

In [ ]:
print(target)

In [ ]:
print(model_feats)

In [ ]:
perc_null(X_train)

In [ ]:
preprocess.fit(data)

In [ ]:
preprocess

In [ ]:

knn = pipeline.fit(X_train, y_train)
print('Training set score: ' + str(knn.score(X_train,y_train)))
print('Test set score: ' + str(knn.score(X_test,y_test)))


In [ ]:
what ... the fuck... it seems there is an infinite value (or too large)